In [2]:
import osparc
import time
from pathlib import Path
from osparc.api import FilesApi, SolversApi
from osparc.models import File, Job, JobInputs, JobOutputs, JobStatus, Solver

print(osparc.__version__)

OSPARC_API_KEY = "ad87a5dc-41b3-547f-90e7-fbc203b55e7c"
OSPARC_API_SECRET = "07c13f52-66cc-5967-ab5e-df66e4a3e3e6"

cfg = osparc.Configuration(
    username=OSPARC_API_KEY,
    password=OSPARC_API_SECRET,
)
print(cfg.host)

with osparc.ApiClient(cfg) as api_client:

    files_api = FilesApi(api_client)
    input_file1: File = files_api.upload_file(file="main.zip")
    input_file2: File = files_api.upload_file(file="INPUT_FOLDER/input.json")

    solvers_api = SolversApi(api_client)
    solver: Solver = solvers_api.get_solver_release(
        "simcore/services/comp/osparc-python-runner", "1.2.0"
    )

    job: Job = solvers_api.create_job(
        solver.id,
        solver.version,
        JobInputs(
            {
                "input_2": input_file2,
                "input_1": input_file1,
            }
        ),
    )

    status: JobStatus = solvers_api.start_job(solver.id, solver.version, job.id)
    while not status.stopped_at:
        time.sleep(3)
        status = solvers_api.inspect_job(solver.id, solver.version, job.id)
        print("Solver progress", f"{status.progress}/100", flush=True)

    outputs: JobOutputs = solvers_api.get_job_outputs(solver.id, solver.version, job.id)

    print(f"Job {outputs.job_id} got these results:")
    for output_name, result in outputs.results.items():
        print(output_name, "=", result)

    results_file: File = outputs.results["output_1"]
    download_path: str = files_api.download_file(file_id=results_file.id)
    print(download_path)

0.4.3
https://api.osparc.io
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 0/100
Solver progress 100/100
Job 5308bd58-738f-48cc-abbe-cae116d42377 got these results:
output_1 = {'checksum': 'd21b442bad1385f3279b5b050bea6609',
 'content_type': 'application/zip',
 'filename': 'output_data.zip',
 'id': '2d9c1e79-138f-34b8-885c-bb587d17f22e'}
C:\Users\Calmi2\AppData\Local\Temp\tmphedtd5r2
